# Introduction

In this project, we will show examples of how we can enable employees to interact with their company's databases using natural language. This means that they no longer have to rely on data professionals to "give" them access to the data they need for making informed business decisions.

We will first create a SQLite database using 17 CSV files, and each CSV file will represent a table in the database. This database is a simplified representation of a real-world database for a typical HR SaaS company.

Next, we will develop a Q&A chatbot powered by OpenAI's GPT4-o and the popular Langchain library. We will also be using an agent framework, which will help the LLM tackle more complex queries.

If you want to run the code yourself, be sure to first upload the 17 CSV files in the directory and enter your secret OpenAI API key below.

In [1]:
# Ensure necessary packages are installed
!pip install langchain
!pip install langchain-community
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import libraries
import sqlite3
import pandas as pd
from tabulate import tabulate
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor, create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain.tools import tool, Tool
import os, json, re, openai, ast
from datetime import datetime
from langchain_community.callbacks import get_openai_callback

In [3]:
# Fill in API key from OpenAI
os.environ["OPENAI_API_KEY"] = "<enter your API key>"

# Langchain tracking
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "<enter your API key or simply remove all Langchain tracking variables>"
os.environ["LANGCHAIN_PROJECT"] = "FAIM project"

In [4]:
# Instantiate LLM and ensure temperature is set to 0 (we don't want the LLM to produce creative output!)
temperature = 0
model = "gpt-4o"

llm = ChatOpenAI(temperature=temperature, model=model)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
# Create SQLite database
conn = sqlite3.connect('DATABASE.db')

# List of CSV files to import
csv_files = [
    'AttendanceRecords.csv',
    'Bonuses.csv',
    'Companies.csv',
    'Deductions.csv',
    'Departments.csv',
    'FeatureUsage.csv',
    'Feedback.csv',
    'Goals.csv',
    'Invoices.csv',
    'LeaveRequests.csv',
    'LeaveTypes.csv',
    'PayrollRecords.csv',
    'PerformanceReviews.csv',
    'Positions.csv',
    'Subscriptions.csv',
    'UserSessions.csv',
    'Employees.csv'
]

# Import each CSV file into a table in the SQLite database
for csv_file in csv_files:
    table_name = csv_file.split('.')[0]
    df = pd.read_csv(csv_file)
    df.to_sql(table_name, conn, if_exists='replace', index=False)

# Close the connection
conn.close()


In [6]:
# Connect to the SQLite database
conn = sqlite3.connect('DATABASE.db')

# List of all table names
table_names = [
    "Companies", "Employees", "Departments", "Positions",
    "LeaveTypes", "LeaveRequests", "AttendanceRecords",
    "PerformanceReviews", "Goals", "PayrollRecords",
    "Deductions", "Bonuses", "FeatureUsage",
    "Feedback", "UserSessions", "Subscriptions", "Invoices"
]

# Fetch and print 10 random rows from each table
def print_random_rows(table_name):
    query = f"SELECT * FROM {table_name} ORDER BY RANDOM() LIMIT 10;"
    df = pd.read_sql_query(query, conn)
    print(f"--- {table_name} ---")
    print(tabulate(df, headers='keys', tablefmt='psql'))
    print("\n")

for table_name in table_names:
    print_random_rows(table_name)

# Close the database connection
conn.close()


--- Companies ---
+----+-------------+------------------------+------------------------+---------------+-----------------+-------------+
|    |   CompanyID | CompanyName            | Industry               | LeadSource    | CustomerSince   | Geography   |
|----+-------------+------------------------+------------------------+---------------+-----------------+-------------|
|  0 |          42 | Ebony Innovations      | Communication Services | Website       | 25/4/23         | Hong Kong   |
|  1 |          50 | Cypress Ventures       | Energy                 | Social Media  | 10/6/23         | Indonesia   |
|  2 |          59 | Walnut Enterprises     | Financials             | Direct        | 16/2/16         | Thailand    |
|  3 |          57 | Hazelnut Innovations   | Financials             | Website       | 23/11/18        | Indonesia   |
|  4 |          44 | Pine Enterprises       | Utilities              | Referral      | 17/6/14         | Singapore   |
|  5 |          76 | Cosmos Ho

In [7]:
# Initialize a wrapper around the SQLite database. In this project, we will give the agent access to all tables.
db = SQLDatabase.from_uri(
    "sqlite:///DATABASE.db",
    include_tables=table_names,
    sample_rows_in_table_info=3,
)

# Setting up tools for our agent

In [8]:
# Use Langchain's SQLDatabaseToolkit to provide our agent with access to a bunch of important SQL tools
# Examples: connecting to database, examining table schema, validating and executing queries
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [9]:
# Create dictionary that returns basic descriptions of each table in our database
TABLES_DESCRIPTIONS = {
    "AttendanceRecords": "Tracks daily attendance of our customers' employees",
    "Bonuses": "Contains information about bonuses awarded to our customers' employees",
    "Companies": "Contains information about our customers. The CompanyID column is the primary key for this table, and is a foreign key in several other tables",
    "Deductions": "Contains information about deductions from our customers' employees payroll",
    "Departments": "Contains information about our customers' departments",
    "Employees": "Contains information about our customers' employees. The EmployeeID column is the primary key for this table, and is a foreign key in several other tables",
    "FeatureUsage": "Contains information about our customers' usage of our platform features",
    "Feedback": "Contains information about our customers' feedback on our platform",
    "Goals": "Contains information about the goals set for our customers' employees",
    "Invoices": "Contains information about invoices issued to our customers",
    "LeaveRequests": "Contains information about leave requests submitted by our customers' employees",
    "LeaveTypes": "Contains information about leave types available for each customer",
    "PayrollRecords": "Contains information about our customers' employees' payroll records",
    "PerformanceReviews": "Contains information about performance reviews submitted by our customers' employees",
    "Positions": "Contains information about all the work positions at each company",
    "Subscriptions": "Contains information about subscriptions purchased by our customers",
    "UserSessions": "Tracks user sessions to analyse login frequency and session duration",
}

def get_tables_descriptions(query: str) -> str:

    return json.dumps(TABLES_DESCRIPTIONS)

# Create function that returns today's date
def get_today_date(query: str) -> str:

    # Getting today's date in string format
    today_date_string = datetime.now().strftime("%Y-%m-%d")
    return today_date_string

# Create function that runs a query and saves the results in a list
def run_query_save_results(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub]
    return res

# Create function to get distinct values from each text column in Employees table
def get_distinct_values_from_employees(query: str) -> str:

    nat = run_query_save_results(db,"SELECT DISTINCT Nationality FROM Employees")

    nat_str = (
        "List of unique values of the Nationality column : \n"
        + json.dumps(nat)
    )

    gen = run_query_save_results(db,"SELECT DISTINCT Gender FROM Employees")

    gen_str = (
        "List of unique values of the Gender column : \n"
        + json.dumps(gen)
    )

    stat = run_query_save_results(db,"SELECT DISTINCT EmploymentStatus FROM Employees")

    stat_str = (
        "List of unique values of the EmploymentStatus column : \n"
        + json.dumps(stat)
    )

    return nat_str + gen_str + stat_str


# Create function to get distinct values from each text column in Companies table
def get_distinct_values_from_companies(query: str) -> str:

    ind = run_query_save_results(db,"SELECT DISTINCT Industry FROM Companies")

    ind_str = (
        "List of unique values of the Industry column : \n"
        + json.dumps(ind)
    )

    lead = run_query_save_results(db,"SELECT DISTINCT LeadSource FROM Companies")

    lead_str = (
        "List of unique values of the LeadSource column : \n"
        + json.dumps(lead)
    )

    geo = run_query_save_results(db,"SELECT DISTINCT Geography FROM Companies")

    geo_str = (
        "List of unique values of the Geography column : \n"
        + json.dumps(geo)
    )

    return ind_str + lead_str + geo_str


# Create function to get distinct values from each text column in FeatureUsage table
def get_distinct_values_from_FeatureUsage(query: str) -> str:

    mod = run_query_save_results(db,"SELECT DISTINCT ModuleName FROM FeatureUsage")

    mod_str = (
        "List of unique values of the ModuleName column : \n"
        + json.dumps(mod)
    )

    tas = run_query_save_results(db,"SELECT DISTINCT TaskDescription FROM FeatureUsage")

    tas_str = (
        "List of unique values of the TaskDescription column : \n"
        + json.dumps(tas)
    )

    com = run_query_save_results(db,"SELECT DISTINCT CompletionStatus FROM FeatureUsage")

    com_str = (
        "List of unique values of the CompletionStatus column : \n"
        + json.dumps(com)
    )

    return mod_str + tas_str + com_str


# Create function to get distinct values from each text column in Feedback table
def get_distinct_values_from_Feedback(query: str) -> str:

    mod = run_query_save_results(db,"SELECT DISTINCT ModuleName FROM Feedback")

    mod_str = (
        "List of unique values of the ModuleName column : \n"
        + json.dumps(mod)
    )

    fed = run_query_save_results(db,"SELECT DISTINCT FeedbackType FROM Feedback")

    fed_str = (
        "List of unique values of the FeedbackType column : \n"
        + json.dumps(fed)
    )

    return mod_str + fed_str


# Create function to get distinct values from each text column in LeaveRequests table
def get_distinct_values_from_LeaveRequests(query: str) -> str:

    req = run_query_save_results(db,"SELECT DISTINCT RequestStatus FROM LeaveRequests")

    req_str = (
        "List of unique values of the RequestStatus column : \n"
        + json.dumps(req)
    )

    return req_str


# Create function to get distinct values from each text column in Subscriptions table
def get_distinct_values_from_Subscriptions(query: str) -> str:

    sta = run_query_save_results(db,"SELECT DISTINCT Status FROM Subscriptions")

    sta_str = (
        "List of unique values of the Status column : \n"
        + json.dumps(sta)
    )

    return sta_str



In [10]:
# Use the functions above to create extra tools for our agent (in addition to the main SQL toolkit)
def extra_tools():
    tools = [
        Tool.from_function(
            func=get_tables_descriptions,
            name="get_tables_descriptions",
            description="Useful to get basic information about the contents of each table. A json is returned.",
        ),
        Tool.from_function(
            func=get_today_date,
            name="get_today_date",
            description="""Useful to get today's date. This is useful when the user needs to obtain information
            pertaining to a certain date range. For example, the user might want to obtain data for the last 6 months.""",
        ),
        Tool.from_function(
            func=get_distinct_values_from_employees,
            name="get_distinct_values_from_employees",
            description="Use this to get unique values for selected text columns in the Employees table.",
        ),
        Tool.from_function(
            func=get_distinct_values_from_companies,
            name="get_distinct_values_from_companies",
            description="Use this to get unique values for selected text columns in the Companies table.",
        ),
        Tool.from_function(
            func=get_distinct_values_from_FeatureUsage,
            name="get_distinct_values_from_FeatureUsage",
            description="Use this to get unique values for selected text columns in the FeatureUsage table.",
        ),
        Tool.from_function(
            func=get_distinct_values_from_Feedback,
            name="get_distinct_values_from_Feedback",
            description="Use this to get unique values for selected text columns in the Feedback table.",
        ),
        Tool.from_function(
            func=get_distinct_values_from_LeaveRequests,
            name="get_distinct_values_from_LeaveRequests",
            description="Use this to get unique values for selected text columns in the LeaveRequests table.",
        ),
        Tool.from_function(
            func=get_distinct_values_from_Subscriptions,
            name="get_distinct_values_from_Subscriptions",
            description="Use this to get unique values for selected text columns in the Subscriptions table.",
        ),
    ]
    return tools

extra_tools = extra_tools()

In [11]:
# Create a prompt template for the agent's LLM prompt. Give the agent context and directions on how to think about the user’s question.
prompt_template = """
You are a world class data analyst at a HR SaaS startup called Omni. Omni's customers are companies based in Asia
who subscribe to its platform to help them better manage their HR needs.

Your role is to help your colleagues from Omni interact with the company's SQL database using natural language.

You are given a bunch of tools that enable you to perform a variety of tasks. For example, you can transform
the user's question from natural language to SQL code, connect with the company's database and run queries,
get information about each table in the database as well as getting the date today.

No matter what your colleague's question is, you must always call the tool 'get_tables_descriptions' at the start
to understand about the contents of each table in the database.

In queries involving string or text comparisons, you should use the `LIKE` operator for fuzzy matching
and / or the `LOWER` operator for case-insensitive comparisons. In these situations, you should
also first check for a tool that allows you to obtain all unique values in that column. If a tool like this exists,
using it will allow you to identify the range of possible values in that column.

In queries involving company names from the Companies table, you should only use the `LOWER` operator for case-insensitive
comparisons. Never, ever use the `LIKE`operator!

Whenever possible, you should try to use the output of the executed query to answer your colleague's question. If you do
not know the answer, do not fabricate a response. Simply say you do not know.

If your colleague's question is ambiguous, you should get her to clarify what she means. Do not make assumptions.
For example, if your colleague asks who the top 3 customers are, you should ask what she means by top 3 -
is it based on billing amount, number of employees or something else?

You should always be polite and respectful in all of your interactions.

Your colleague's question: {input_question}

Your response:
"""

In [12]:
# Create function to run agent with an input question
def run_agent_with_input(input_question):
    # Create the prompt with the input question
    prompt = prompt_template.format(input_question=input_question)

    # Create agent
    agent = create_sql_agent(
        llm=llm,
        toolkit=toolkit,
        agent_type="openai-tools",
        extra_tools=extra_tools,
        handle_parsing_errors=True,
        verbose=True,
        suffix=prompt,
    )

    # Run the agent with the input
    response = agent.invoke(input_question)

    return response

# Query examples

You can try out your own questions by changing the value of the question variable

In [13]:
question = "how many customers are based in indonesia or MALAYSIA?"
response = run_agent_with_input(question)

# Display response from LLM
print(f"\n {response['output']}")



> Entering new SQL Agent Executor chain...

Invoking: `get_tables_descriptions` with `Companies`


{"AttendanceRecords": "Tracks daily attendance of our customers' employees", "Bonuses": "Contains information about bonuses awarded to our customers' employees", "Companies": "Contains information about our customers. The CompanyID column is the primary key for this table, and is a foreign key in several other tables", "Deductions": "Contains information about deductions from our customers' employees payroll", "Departments": "Contains information about our customers' departments", "Employees": "Contains information about our customers' employees. The EmployeeID column is the primary key for this table, and is a foreign key in several other tables", "FeatureUsage": "Contains information about our customers' usage of our platform features", "Feedback": "Contains information about our customers' feedback on our platform", "Goals": "Contains information about the goals set for our customers

Cost in USD (obtained from LangSmith):

9,303 prompt tokens / $0.046515

66 completion tokens / $0.00099

In [ ]:
question = "How many of our customers have Korean employees"
response = run_agent_with_input(question)

# Display response from LLM
print(f"\n {response['output']}")



> Entering new SQL Agent Executor chain...

Invoking: `get_tables_descriptions` with `Employees, Companies`


{"AttendanceRecords": "Tracks daily attendance of our customers' employees", "Bonuses": "Contains information about bonuses awarded to our customers' employees", "Companies": "Contains information about our customers. The CompanyID column is the primary key for this table, and is a foreign key in several other tables", "Deductions": "Contains information about deductions from our customers' employees payroll", "Departments": "Contains information about our customers' departments", "Employees": "Contains information about our customers' employees. The EmployeeID column is the primary key for this table, and is a foreign key in several other tables", "FeatureUsage": "Contains information about our customers' usage of our platform features", "Feedback": "Contains information about our customers' feedback on our platform", "Goals": "Contains information about the goals set for ou

4,887 prompt tokens / $0.024435

39 completion tokens / $0.000585

In [ ]:
question = """Give me a breakdown of all customers by acquisition source. For example,
i want to know what percentage of customers came from referrals, socials and so on"""

response = run_agent_with_input(question)

# Display response from LLM
print(f"\n {response['output']}")



> Entering new SQL Agent Executor chain...

Invoking: `get_tables_descriptions` with `Companies`


{"AttendanceRecords": "Tracks daily attendance of our customers' employees", "Bonuses": "Contains information about bonuses awarded to our customers' employees", "Companies": "Contains information about our customers. The CompanyID column is the primary key for this table, and is a foreign key in several other tables", "Deductions": "Contains information about deductions from our customers' employees payroll", "Departments": "Contains information about our customers' departments", "Employees": "Contains information about our customers' employees. The EmployeeID column is the primary key for this table, and is a foreign key in several other tables", "FeatureUsage": "Contains information about our customers' usage of our platform features", "Feedback": "Contains information about our customers' feedback on our platform", "Goals": "Contains information about the goals set for our customers

5,283 prompt tokens / $0.026415

92 completion tokens / $0.00138

In [ ]:
question = """I am the account manager for bisnis indo. Can you find out what proportion of
          their employees are part time"""

response = run_agent_with_input(question)

# Display response from LLM
print(f"\n {response['output']}")



> Entering new SQL Agent Executor chain...

Invoking: `get_tables_descriptions` with `Employees, Companies`


{"AttendanceRecords": "Tracks daily attendance of our customers' employees", "Bonuses": "Contains information about bonuses awarded to our customers' employees", "Companies": "Contains information about our customers. The CompanyID column is the primary key for this table, and is a foreign key in several other tables", "Deductions": "Contains information about deductions from our customers' employees payroll", "Departments": "Contains information about our customers' departments", "Employees": "Contains information about our customers' employees. The EmployeeID column is the primary key for this table, and is a foreign key in several other tables", "FeatureUsage": "Contains information about our customers' usage of our platform features", "Feedback": "Contains information about our customers' feedback on our platform", "Goals": "Contains information about the goals set for ou

5,160 prompt tokens / $0.0258

166 completion tokens / $0.00249

In [ ]:
question = "Which customer from the property sector has the most employees? How many employees does it have?"

response = run_agent_with_input(question)

# Display response from LLM
print(f"\n {response['output']}")



> Entering new SQL Agent Executor chain...

Invoking: `get_tables_descriptions` with `Companies, Employees`


{"AttendanceRecords": "Tracks daily attendance of our customers' employees", "Bonuses": "Contains information about bonuses awarded to our customers' employees", "Companies": "Contains information about our customers. The CompanyID column is the primary key for this table, and is a foreign key in several other tables", "Deductions": "Contains information about deductions from our customers' employees payroll", "Departments": "Contains information about our customers' departments", "Employees": "Contains information about our customers' employees. The EmployeeID column is the primary key for this table, and is a foreign key in several other tables", "FeatureUsage": "Contains information about our customers' usage of our platform features", "Feedback": "Contains information about our customers' feedback on our platform", "Goals": "Contains information about the goals set for ou

9,840 prompt tokens / $0.0492

149 completion tokens / $0.002235

In [ ]:
question = """Show me the average feedback score for each module in the first half of 2024. How has it changed compared to
          the second half of 2023?"""
response = run_agent_with_input(question)

# Display response from LLM
print(f"\n {response['output']}")



> Entering new SQL Agent Executor chain...

Invoking: `get_tables_descriptions` with `Feedback`


{"AttendanceRecords": "Tracks daily attendance of our customers' employees", "Bonuses": "Contains information about bonuses awarded to our customers' employees", "Companies": "Contains information about our customers. The CompanyID column is the primary key for this table, and is a foreign key in several other tables", "Deductions": "Contains information about deductions from our customers' employees payroll", "Departments": "Contains information about our customers' departments", "Employees": "Contains information about our customers' employees. The EmployeeID column is the primary key for this table, and is a foreign key in several other tables", "FeatureUsage": "Contains information about our customers' usage of our platform features", "Feedback": "Contains information about our customers' feedback on our platform", "Goals": "Contains information about the goals set for our customers'

5,888 prompt tokens / $0.02944

283 completion tokens / $0.004245

In [ ]:
question = """Which customer from HK used payroll module most frequently last week?"""

response = run_agent_with_input(question)

# Display response from LLM
print(f"\n {response['output']}")



> Entering new SQL Agent Executor chain...

Invoking: `get_tables_descriptions` with `Companies, FeatureUsage`


{"AttendanceRecords": "Tracks daily attendance of our customers' employees", "Bonuses": "Contains information about bonuses awarded to our customers' employees", "Companies": "Contains information about our customers. The CompanyID column is the primary key for this table, and is a foreign key in several other tables", "Deductions": "Contains information about deductions from our customers' employees payroll", "Departments": "Contains information about our customers' departments", "Employees": "Contains information about our customers' employees. The EmployeeID column is the primary key for this table, and is a foreign key in several other tables", "FeatureUsage": "Contains information about our customers' usage of our platform features", "Feedback": "Contains information about our customers' feedback on our platform", "Goals": "Contains information about the goals set for

17,715 prompt tokens / $0.088575

323 completion tokens / $0.004845

In [ ]:
question = """What is the average session duration for cv asia's employees for each month in 2024"""

response = run_agent_with_input(question)

# Display response from LLM
print(f"\n {response['output']}")



> Entering new SQL Agent Executor chain...

Invoking: `get_tables_descriptions` with `Employees, Companies, Sessions`


{"AttendanceRecords": "Tracks daily attendance of our customers' employees", "Bonuses": "Contains information about bonuses awarded to our customers' employees", "Companies": "Contains information about our customers. The CompanyID column is the primary key for this table, and is a foreign key in several other tables", "Deductions": "Contains information about deductions from our customers' employees payroll", "Departments": "Contains information about our customers' departments", "Employees": "Contains information about our customers' employees. The EmployeeID column is the primary key for this table, and is a foreign key in several other tables", "FeatureUsage": "Contains information about our customers' usage of our platform features", "Feedback": "Contains information about our customers' feedback on our platform", "Goals": "Contains information about the goals 

13,379 prompt tokens / $0.066895

168 completion tokens / $0.00252

In [ ]:
question = """Which IT companies from malaysia have subscription plans worth more than $1000"""
response = run_agent_with_input(question)

# Display response from LLM
print(f"\n {response['output']}")



> Entering new SQL Agent Executor chain...

Invoking: `get_tables_descriptions` with `Companies, Subscriptions`


{"AttendanceRecords": "Tracks daily attendance of our customers' employees", "Bonuses": "Contains information about bonuses awarded to our customers' employees", "Companies": "Contains information about our customers. The CompanyID column is the primary key for this table, and is a foreign key in several other tables", "Deductions": "Contains information about deductions from our customers' employees payroll", "Departments": "Contains information about our customers' departments", "Employees": "Contains information about our customers' employees. The EmployeeID column is the primary key for this table, and is a foreign key in several other tables", "FeatureUsage": "Contains information about our customers' usage of our platform features", "Feedback": "Contains information about our customers' feedback on our platform", "Goals": "Contains information about the goals set fo

7,794 prompt tokens / $0.03897

122 completion tokens / $0.00183

In [ ]:
question = """Give a breakdown of the leave requests submitted in the past 10 days by status"""

response = run_agent_with_input(question)

# Display response from LLM
print(f"\n {response['output']}")



> Entering new SQL Agent Executor chain...

Invoking: `get_today_date` with ``


2024-05-27
Invoking: `sql_db_list_tables` with `{}`


AttendanceRecords, Bonuses, Companies, Deductions, Departments, Employees, FeatureUsage, Feedback, Goals, Invoices, LeaveRequests, LeaveTypes, PayrollRecords, PerformanceReviews, Positions, Subscriptions, UserSessions
Invoking: `sql_db_schema` with `{'table_names': 'LeaveRequests'}`



CREATE TABLE "LeaveRequests" (
	"LeaveRequestID" INTEGER, 
	"EmployeeID" INTEGER, 
	"LeaveTypeID" INTEGER, 
	"StartDate" TEXT, 
	"EndDate" TEXT, 
	"RequestStatus" TEXT, 
	"RequestDate" TEXT, 
	"ApproverID" INTEGER, 
	"LeaveReason" TEXT, 
	"LeaveDuration" INTEGER
)

/*
3 rows from LeaveRequests table:
LeaveRequestID	EmployeeID	LeaveTypeID	StartDate	EndDate	RequestStatus	RequestDate	ApproverID	LeaveReason	LeaveDuration
1	16	3	2024-05-07	2024-05-15	Pending	2023-11-08	48	Yoga retreat	8
2	29	5	2023-12-01	2023-12-01	Rejected	2022-10-27	49	Cruise	0
3	42	5	2022-07-29	2022-08-01

5,180 prompt tokens / $0.0259

79 completion tokens / $0.001185